In [1]:
import pandas as pd
import numpy as np
import scipy.signal as signal
from scipy import stats
from biosppy.signals import ecg
import neurokit2 as nk
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
import warnings

warnings.filterwarnings('ignore')

In [2]:

train_data = pd.read_csv('train.csv')

train_labels = train_data['y']
train_data = train_data.drop(columns=['id', 'y'])


In [3]:

def hjorth_parameters(signal):
    first_derivative = np.diff(signal)
    second_derivative = np.diff(signal, n=2)
    var_zero = np.var(signal)
    var_d1 = np.var(first_derivative)
    var_d2 = np.var(second_derivative)
    if var_zero == 0 or var_d1 == 0 or var_d2 == 0:
        return np.nan, np.nan, np.nan
    activity = var_zero
    mobility = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1)
    return activity, mobility, complexity


In [4]:
# Feature Extraction

counter=0

def extract_features(row, sampling_rate=300):
    # drop NaN values
    global counter
    signal_raw = row.dropna().values
    
    if len(signal_raw) < sampling_rate * 2:  # Ensure at least 2 seconds of data
        print("Signal too short, skipping feature extraction.")
        counter= counter +1
        return features

    # a dictionary to store features of the current signal
    features = {}

    try:

        # Noise Reduction
        
        # Bandpass Filter between 0.5 Hz and 40 Hz ? to recheck 
        nyquist = 0.5 * sampling_rate
        low = 0.5 / nyquist
        high = 40 / nyquist
        b, a = signal.butter(5, [low, high], btype='band')
        signal_filtered = signal.filtfilt(b, a, signal_raw)
    
        # Denoising using ECG Clean function
        signal_denoised = nk.ecg_clean(signal_filtered, sampling_rate=sampling_rate, method='neurokit')
        signal_denoised = signal_denoised[~np.isnan(signal_denoised)]
        # *************** function nk.ecg_process does the cleaning too. should I remove this one, or is it still necessary????

        r_peaks_m = ecg.engzee_segmenter(signal_denoised, 300)['rpeaks']
        beats = ecg.extract_heartbeats(signal_denoised, r_peaks_m, 300)['templates']
        mu = np.mean(beats, axis=0) 
        
        # Basic Statistical Features
        features['signal_mean'] = np.mean(signal_denoised)
        features['signal_std'] = np.std(signal_denoised) 
        features['signal_min'] = np.min(signal_denoised)
        features['signal_max'] = np.max(signal_denoised)
        features['signal_skewness'] = stats.skew(signal_denoised) # Should be on the average heartbeat
        features['signal_kurtosis'] = stats.kurtosis(signal_denoised) 
    
        # Hjorth Parameters
        activity, mobility, complexity = hjorth_parameters(signal_denoised)
        features['hjorth_activity'] = activity
        features['hjorth_mobility'] = mobility
        features['hjorth_complexity'] = complexity
    
    
        # Proceed with advanced features only if the signal is long enough
        if len(signal_denoised) >= sampling_rate * 2:
            # ECG Processing using NeuroKit2
            ecg_signals, ecg_info = nk.ecg_process(signal_denoised, sampling_rate=sampling_rate)
    
            r_peaks = ecg_info['ECG_R_Peaks']
    
            if len(r_peaks) >= 4:  # Need at least 4 R-peaks for HRV analysis
                rri = np.diff(r_peaks) / sampling_rate * 1000 
                features['RR_mean'] = np.mean(rri) # is it the same as what is given by the nk.hrv_time ?
                
                # Time-Domain Features
                hrv_time = nk.hrv_time(r_peaks, sampling_rate=sampling_rate, show=False)
                features.update(hrv_time.iloc[0].to_dict())
    
                # Frequency-Domain Features
                hrv_freq = nk.hrv_frequency(r_peaks, sampling_rate=sampling_rate, show=False)
                features.update(hrv_freq.iloc[0].to_dict())
    
                # Non-linear Features
                # Only if RR intervals are sufficient
                if len(rri) >= 10:
                    try:
                        hrv_nonlinear = nk.hrv_nonlinear(r_peaks, sampling_rate=sampling_rate, show=False)
                        features.update(hrv_nonlinear.iloc[0].to_dict())
                    except ValueError as e:
                        counter = counter +1
                        #print(f"Skipping non-linear features for this signal due to: {e}")
    
                # Morphological Features
                # PQRST features using Biosppy
                ecg_out = ecg.ecg(signal=signal_denoised, sampling_rate=sampling_rate, show=False)
                templates = ecg_out['templates']
                heart_rate = ecg_out['heart_rate']
    
                if templates.size > 0 and heart_rate.size > 0:
                    # Mean and STD of heart rate
                    features['HR_mean'] = np.mean(heart_rate)
                    features['HR_std'] = np.std(heart_rate)
    
                    # Mean and STD of QRS complexes
                    features['QRS_mean'] = np.mean(templates)
                    features['QRS_std'] = np.std(templates)
    
                # Calculate PR,QT, and QRS durations
                
                delineate_signal, delineate_info = nk.ecg_delineate(
                    signal_denoised, r_peaks, sampling_rate=sampling_rate, method="dwt", show=False)
    
                required_keys = ['ECG_P_Onsets', 'ECG_R_Onsets', 'ECG_T_Offsets',
                                 'ECG_R_Offsets', 'ECG_T_Onsets', 'ECG_P_Peaks', 'ECG_T_Peaks']
                if all(key in delineate_info and len(delineate_info[key]) > 0 for key in required_keys):
                    # PR Interval
                    pr_interval = ( np.array(delineate_info['ECG_R_Onsets']) - np.array(delineate_info['ECG_P_Onsets'])) / sampling_rate
                    features['PR_interval_mean'] = np.mean(pr_interval)
                    features['PR_interval_std'] = np.std(pr_interval)
    
                    # QT Interval
                    qt_interval = ( np.array(delineate_info['ECG_T_Offsets']) - np.array(delineate_info['ECG_R_Onsets'])) / sampling_rate
                    features['QT_interval_mean'] = np.mean(qt_interval)
                    features['QT_interval_std'] = np.std(qt_interval)
    
                    # QRS Duration
                    qrs_duration = (np.array(delineate_info['ECG_R_Offsets']) - np.array(delineate_info['ECG_R_Onsets'])) / sampling_rate
                    features['QRS_duration_mean'] = np.mean(qrs_duration)
                    features['QRS_duration_std'] = np.std(qrs_duration)
    
                    # ST Segment
                    st_segment = (np.array(delineate_info['ECG_T_Onsets']) - np.array(delineate_info['ECG_R_Offsets'])) / sampling_rate
                    features['ST_segment_mean'] = np.mean(st_segment)
                    features['ST_segment_std'] = np.std(st_segment)
    
                    # T-wave amplitude
                    ecg_t_peaks = np.array(delineate_info['ECG_T_Peaks'])
                    ecg_t_peaks = ecg_t_peaks[~np.isnan(ecg_t_peaks)]
                    ecg_t_peaks = ecg_t_peaks.astype(int)
                    
                    t_waves = signal_denoised[ecg_t_peaks]
                    features['T_wave_amplitude_mean'] = np.mean(t_waves)
                    features['T_wave_amplitude_std'] = np.std(t_waves)
    
                    
                    # P-wave amplitude
                    ecg_p_peaks = np.array(delineate_info['ECG_P_Peaks'])
                    ecg_p_peaks = ecg_p_peaks[~np.isnan(ecg_p_peaks)]
                    ecg_p_peaks = ecg_p_peaks.astype(int)
                    
                    p_waves = signal_denoised[ecg_p_peaks]
                    features['P_wave_amplitude_mean'] = np.mean(p_waves)
                    features['P_wave_amplitude_std'] = np.std(p_waves)
    
                    # R-wave amplitude
                    r_waves = signal_denoised[r_peaks]
                    features['R_wave_amplitude_mean'] = np.mean(r_waves)
                    features['R_wave_amplitude_std'] = np.std(r_waves)
        
    except Exception as e:
        counter = counter +1

    return features

In [5]:
# Feature Extraction for Training Data

sampling_rate = 300  # Hz
train_features = []

for index, row in train_data.iterrows():
    features = extract_features(row, sampling_rate=sampling_rate)
    train_features.append(features)

# Convert the list of feature dictionaries to a DataFrame
train_feature_df = pd.DataFrame(train_features)
proportion_of_missing_values= train_feature_df.isnull().mean()
columns_to_drop = proportion_of_missing_values[proportion_of_missing_values>0.2].index
print(f'Not fully prerocessed datapoints: {counter}')
print(f'columns to drop: {len(columns_to_drop)}')
train_feature_df= train_feature_df.drop(columns=columns_to_drop)


Not fully prerocessed datapoints: 68
columns to drop: 31


In [6]:
#replace nan values
train_feature_df = pd.DataFrame(train_feature_df)
train_feature_df.replace([np.inf, -np.inf], np.nan, inplace=True)
train_feature_df.fillna(train_feature_df.mean(), inplace=True)

#Standardize the data
scaler = preprocessing.StandardScaler()
scaler.fit(train_feature_df)
train_feature_df= scaler.transform(train_feature_df)

#dimensionality reduction
train_feature_df= SelectKBest(k=30).fit_transform(train_feature_df, train_labels)

train_feature_df = pd.DataFrame(train_feature_df)
train_labels = train_labels.reset_index(drop=True)
train_feature_df = train_feature_df.reset_index(drop=True)

In [8]:

test_data = pd.read_csv('test.csv')
test_ids = test_data['id']
test_data = test_data.drop(columns=['id'])

# Feature Extraction for Test Data
test_features = []

for index, row in test_data.iterrows():
    features = extract_features(row, sampling_rate=sampling_rate)
    test_features.append(features)

# Convert the list of feature dictionaries to a DataFrame
test_feature_df = pd.DataFrame(test_features)

test_feature_df= test_feature_df.drop(columns=columns_to_drop)
test_feature_df = pd.DataFrame(test_feature_df)
#deal with inf
test_feature_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_feature_df.fillna(test_feature_df.mean(), inplace=True)
#Scaling
test_feature_df= scaler.transform(test_feature_df)
test_feature_df = pd.DataFrame(test_feature_df)


# Ensure the test set has the same features as the training set
missing_cols = set(train_feature_df.columns) - set(test_feature_df.columns)
for c in missing_cols:
    test_feature_df[c] = np.nan

extra_cols = set(test_feature_df.columns) - set(train_feature_df.columns)
test_feature_df = test_feature_df.drop(columns=extra_cols)

test_feature_df = test_feature_df[train_feature_df.columns]


test_feature_df = pd.DataFrame(test_feature_df)

# Reset indices
test_ids = test_ids.reset_index(drop=True)
test_feature_df = test_feature_df.reset_index(drop=True)




In [9]:
# ==================================
# Ensure Feature DataFrames Are Not Empty
if train_feature_df.empty or test_feature_df.empty:
    raise ValueError("Problem, check again the extraction.")

if train_feature_df.shape[1] == 0:
    raise ValueError("No features. check feature extraction.")


test_ids.to_csv('test_ids.csv')
test_feature_df.to_csv('test_features.csv')
train_labels.to_csv('train_labels.csv')
train_feature_df.to_csv('train_features.csv')
